# *Installing Required Libraries*

In [1]:
!pip install transformers
!pip install datasets
!pip install accelerate
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylin

# *Importing libraries*

In [2]:
import json
import wandb
import pandas as pd
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import torch
from datasets import Dataset
from datasets import load_dataset
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForMaskedLM,AutoModelForSequenceClassification
from transformers import TrainerCallback, EarlyStoppingCallback
from transformers import TrainingArguments, Trainer

In [3]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
val_df = pd.read_csv("val.csv")


train_df = train_df.drop(columns=['Num_Tokens','Num_Sentences','corrected_text'])
test_df = test_df.drop(columns=['Num_Tokens','Num_Sentences','corrected_text'])
val_df = val_df.drop(columns=['Num_Tokens','Num_Sentences','corrected_text'])

In [4]:
train_df['labels'].value_counts()

,count
labels,
0,2319
1,741


In [5]:
val_df['labels'].value_counts()

,count
labels,
0,774
1,247


In [6]:
test_df['labels'].value_counts()

,count
labels,
0,774
1,247


In [7]:
train_df.to_csv('final_train.csv',index=False, encoding='utf-8')
test_df.to_csv('final_test.csv',index=False, encoding='utf-8')
val_df.to_csv('final_val.csv',index=False, encoding='utf-8')

## *Dataset loading as HuggingFace Trainer*

In [8]:
dataset = load_dataset('csv', data_files={'train': "final_train.csv",
                                              'val':"final_val.csv",'test':"final_test.csv"})

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['clean_text', 'labels', 'language_tags'],
        num_rows: 3060
    })
    val: Dataset({
        features: ['clean_text', 'labels', 'language_tags'],
        num_rows: 1021
    })
    test: Dataset({
        features: ['clean_text', 'labels', 'language_tags'],
        num_rows: 1021
    })
})

In [10]:
import pickle
with open('/content/rh-code-mixed-2.pkl', 'rb') as f:
    related_words = pickle.load(f)

## Interleaved Word-Language Approach

In [11]:
def interleaved_preprocess(sentences):
    texts = sentences['clean_text']
    tags = sentences['language_tags']

    processed_texts = []

    for text, tag in zip(texts, tags):
        words = text.split()
        tags_list = tag.split(", ")
        interleaved_text = " ".join([f"{related_words[word].get('corrected_word',word)} [{tag}]" for word, tag in zip(words, tags_list)])
        processed_texts.append(interleaved_text)

    return {"interleaved_text_tags": processed_texts}

dataset = dataset.map(interleaved_preprocess, batched=True)

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

In [12]:
dataset['train'][0]

{'clean_text': 'hindu to kuch hai par tum to pori duniya mai gandgi macha rhe ho or bat rhi bhagwe ki to tumhe aba ke abe bhagwa hi the talvar ke dar se salvar khol di unhone isme tumhari koi galti nhi hai',
 'labels': 1,
 'language_tags': 'EN, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, EN, EN, HI, HI, HI, HI, HI, EN, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI',
 'interleaved_text_tags': 'hindu [EN] to [HI] kuch [HI] he [HI] par [HI] tum [HI] to [HI] puri [HI] duniya [HI] may [HI] gandagi [HI] machaa [HI] rahe [HI] ho [HI] or [EN] bat [EN] rahi [HI] bhagave [HI] kii [HI] to [HI] tumhe [HI] aba [EN] ke [HI] abe [HI] bhagava [HI] hi [HI] the [HI] talwaar [HI] ke [HI] dar [HI] se [HI] salwar [HI] khol [HI] di [HI] unhone [HI] isamen [HI] tumhaari [HI] koi [HI] galati [HI] nahin [HI] he [HI]'}

In [13]:
dataset['val'][0]

{'clean_text': 'hindu rastra ke ruzhan miya bhai ko kast hai chalo hindu is bat pe mast hai',
 'labels': 1,
 'language_tags': 'EN, EN, HI, HI, HI, HI, HI, HI, HI, HI, EN, HI, EN, HI, EN, HI',
 'interleaved_text_tags': 'hindu [EN] rastra [EN] ke [HI] rujhaan [HI] mia [HI] bhai [HI] ko [HI] cast [HI] he [HI] chalo [HI] hindu [EN] is [HI] bat [EN] pe [HI] mast [EN] he [HI]'}

In [14]:
dataset['test'][0]

{'clean_text': 'jai hind , in namuno ko bat bhi karni nahi ati hai , inko pension chahiye',
 'labels': 0,
 'language_tags': 'HI, EN, OOV, EN, HI, HI, EN, HI, HI, HI, HI, HI, OOV, HI, EN, HI',
 'interleaved_text_tags': 'jai [HI] hind [EN] , [OOV] in [EN] namoonon [HI] ko [HI] bat [EN] bhi [HI] kerani [HI] nahin [HI] ati [HI] he [HI] , [OOV] inako [HI] pension [EN] chahie [HI]'}

# For 5000 rows dataset with clean_text

# *Tokenization as per Hing-MBERT*

In [15]:
tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/hing-mbert")
print(len(tokenizer))
special_tokens = ["[EN]", "[HI]", "[OOV]"]
tokenizer.add_tokens(special_tokens)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/327 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

119547


3

In [16]:
len(tokenizer)

119550

In [17]:
mbert_dataset = dataset.map(
    lambda example: tokenizer(example['interleaved_text_tags'],max_length=194,padding='max_length', truncation=True),
    batched=True,
    batch_size=64
)
mbert_dataset = mbert_dataset.remove_columns(["clean_text","language_tags","interleaved_text_tags"])
mbert_dataset.set_format("torch")

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

In [18]:
mbert_dataset['train'][0]

{'labels': tensor(1),
 'input_ids': tensor([   101,  19911,  11460, 119547,  10114, 119548,  15694,  10269, 119548,
          10261, 119548,  10248, 119548,  13055,  10147, 119548,  10114, 119548,
          32385,  10116, 119548,  23145,  14990, 119548,  11387, 119548,  13412,
          34350, 119548, 109664,  17394, 119548,  11859,  11643, 119548,  13173,
         119548,  10345, 119547,  11519, 119547,  11859,  11924, 119548,    170,
          65864,  10612, 119548,  10879,  10116, 119548,  10114, 119548,  13055,
          34927,  10112, 119548,  11357,  10113, 119547,  11163, 119548,  11357,
          10112, 119548,    170,  65864,  10362, 119548,  11520, 119548,  10105,
         119548,  13675,  69689, 119548,  11163, 119548,  12923, 119548,  10126,
         119548,  31119,  29445, 119548,    179,  32631, 119548,  10120, 119548,
          10119,  50198, 119548,  26531,  10136, 119548,  13055,  34927,  21960,
          10116, 119548,  11252,  10116, 119548,  74288,  10325, 119548,  

In [19]:
print("Batch Decode:")
print(tokenizer.batch_decode(mbert_dataset['train'][0]['input_ids']))

Batch Decode:
['[CLS]', 'hin', '##du', '[EN]', 'to', '[HI]', 'ku', '##ch', '[HI]', 'he', '[HI]', 'par', '[HI]', 'tu', '##m', '[HI]', 'to', '[HI]', 'pur', '##i', '[HI]', 'dun', '##iya', '[HI]', 'may', '[HI]', 'gan', '##dagi', '[HI]', 'mach', '##aa', '[HI]', 'ra', '##he', '[HI]', 'ho', '[HI]', 'or', '[EN]', 'bat', '[EN]', 'ra', '##hi', '[HI]', 'b', '##haga', '##ve', '[HI]', 'ki', '##i', '[HI]', 'to', '[HI]', 'tu', '##mh', '##e', '[HI]', 'ab', '##a', '[EN]', 'ke', '[HI]', 'ab', '##e', '[HI]', 'b', '##haga', '##va', '[HI]', 'hi', '[HI]', 'the', '[HI]', 'tal', '##waar', '[HI]', 'ke', '[HI]', 'dar', '[HI]', 'se', '[HI]', 'sal', '##war', '[HI]', 'k', '##hol', '[HI]', 'di', '[HI]', 'un', '##hone', '[HI]', 'isam', '##en', '[HI]', 'tu', '##mh', '##aar', '##i', '[HI]', 'ko', '##i', '[HI]', 'gala', '##ti', '[HI]', 'nahi', '##n', '[HI]', 'he', '[HI]', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD

In [20]:
cls = [tokenizer.cls_token_id]
sep = [tokenizer.sep_token_id]

input_str = dataset['train'][0]['interleaved_text_tags']
# Tokenization happens in a few steps:
input_tokens = tokenizer.tokenize(input_str)
input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
input_ids_special_tokens = cls + input_ids + sep

decoded_str = tokenizer.decode(input_ids_special_tokens)

print("start:                ", input_str)
print("tokenize:             ", input_tokens)
print("convert_tokens_to_ids:", input_ids)
print("add special tokens:   ", input_ids_special_tokens)
print("--------")
print("decode:               ", decoded_str)

start:                 hindu [EN] to [HI] kuch [HI] he [HI] par [HI] tum [HI] to [HI] puri [HI] duniya [HI] may [HI] gandagi [HI] machaa [HI] rahe [HI] ho [HI] or [EN] bat [EN] rahi [HI] bhagave [HI] kii [HI] to [HI] tumhe [HI] aba [EN] ke [HI] abe [HI] bhagava [HI] hi [HI] the [HI] talwaar [HI] ke [HI] dar [HI] se [HI] salwar [HI] khol [HI] di [HI] unhone [HI] isamen [HI] tumhaari [HI] koi [HI] galati [HI] nahin [HI] he [HI]
tokenize:              ['hin', '##du', '[EN]', 'to', '[HI]', 'ku', '##ch', '[HI]', 'he', '[HI]', 'par', '[HI]', 'tu', '##m', '[HI]', 'to', '[HI]', 'pur', '##i', '[HI]', 'dun', '##iya', '[HI]', 'may', '[HI]', 'gan', '##dagi', '[HI]', 'mach', '##aa', '[HI]', 'ra', '##he', '[HI]', 'ho', '[HI]', 'or', '[EN]', 'bat', '[EN]', 'ra', '##hi', '[HI]', 'b', '##haga', '##ve', '[HI]', 'ki', '##i', '[HI]', 'to', '[HI]', 'tu', '##mh', '##e', '[HI]', 'ab', '##a', '[EN]', 'ke', '[HI]', 'ab', '##e', '[HI]', 'b', '##haga', '##va', '[HI]', 'hi', '[HI]', 'the', '[HI]', 'tal', '##waar'

# *Calculating Weights for Labels with Imbalanced Dataset*

In [21]:
# Define all possible class labels
class_labels = np.unique(mbert_dataset['train']['labels'])

# Calculate class weights
labels = mbert_dataset['train']['labels']
class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=labels.numpy())
class_weights = torch.tensor(class_weights, dtype=torch.float)
print(class_weights)

tensor([0.6598, 2.0648])


In [22]:
class_labels

array([0, 1])

# **Weighted Loss Trainer for Imbalanced Datset**

In [23]:
from transformers import TrainingArguments, Trainer
# Define a custom Trainer class to include class weights
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels").to(model.device)  # Ensure labels are on the same device as model
        inputs = {k: v.to(model.device) for k, v in inputs.items()}  # Ensure all inputs are on the same device as model
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # Compute weighted loss
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(model.device))  # Move class_weights to the same device as model
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# *Function: Compute metrics for all labels*

In [24]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)

    # Additional metrics
    recall_micro = recall_score(labels, predictions, average='micro')
    f1_micro = f1_score(labels, predictions, average='micro')

    recall_macro = recall_score(labels, predictions, average='macro')
    f1_macro = f1_score(labels, predictions, average='macro')

    recall_positive = recall_score(labels, predictions, pos_label=1)
    f1_positive = f1_score(labels, predictions, pos_label=1)

    recall_negative = recall_score(labels, predictions, pos_label=0)
    f1_negative = f1_score(labels, predictions, pos_label=0)

    return {
        'accuracy': accuracy,
        'precision_weighted': precision,
        'recall_weighted': recall,
        'recall_micro': recall_micro,
        'recall_macro': recall_macro,
        'recall_positive': recall_positive,
        'recall_negative': recall_negative,
        'f1_weighted': f1,
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        'f1_positive': f1_positive,
        'f1_negative': f1_negative
    }

In [ ]:
!export CUDA_LAUNCH_BLOCKING=1

In [25]:
wandb.init(project="rh4", name="Hing_mBERT_itag")
model = AutoModelForSequenceClassification.from_pretrained("l3cube-pune/hing-mbert", num_labels=2)

model.resize_token_embeddings(len(tokenizer))

arguments = TrainingArguments(
    output_dir="sample_HingMBert_trainer_5k_itags",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    optim = 'adamw_torch',
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=5e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1_weighted',  # Define the metric for early stopping
    greater_is_better=True,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224,
    report_to="wandb"
)


trainer = WeightedLossTrainer(
    model=model,
    args=arguments,
    train_dataset=mbert_dataset['train'],
    eval_dataset=mbert_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/712M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/hing-mbert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# *Logger Function for early Stopping and logging*

In [26]:
class LoggingCallback(TrainerCallback):
    def __init__(self, log_path):
        self.log_path = log_path
    # will call on_log on each logging step, specified by TrainerArguement. (i.e TrainerArguement.logginng_step)
    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("total_flos", None)
        if state.is_local_process_zero:
            with open(self.log_path, "a") as f:
                f.write(json.dumps(logs) + "\n")


In [27]:
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.01))
trainer.add_callback(LoggingCallback("sample_HingMBert_trainer_5k_itags/log.jsonl"))

In [28]:
# train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,Recall Micro,Recall Macro,Recall Positive,Recall Negative,F1 Weighted,F1 Micro,F1 Macro,F1 Positive,F1 Negative
1,No log,0.684483,0.760039,0.711116,0.760039,0.760039,0.535749,0.101215,0.970284,0.692770,0.760039,0.514626,0.169492,0.859760


Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,Recall Micro,Recall Macro,Recall Positive,Recall Negative,F1 Weighted,F1 Micro,F1 Macro,F1 Positive,F1 Negative
1,No log,0.684483,0.760039,0.711116,0.760039,0.760039,0.535749,0.101215,0.970284,0.692770,0.760039,0.514626,0.169492,0.859760
2,No log,0.552075,0.686582,0.792919,0.686582,0.686582,0.725745,0.801619,0.649871,0.708934,0.686582,0.655873,0.553073,0.758673
3,No log,0.515542,0.707150,0.808468,0.707150,0.707150,0.748959,0.829960,0.667959,0.727935,0.707150,0.676987,0.578279,0.775694
4,No log,0.504961,0.793340,0.812692,0.793340,0.793340,0.758945,0.692308,0.825581,0.800270,0.793340,0.738370,0.618445,0.858294
5,No log,0.599470,0.799216,0.801193,0.799216,0.799216,0.731120,0.599190,0.863049,0.800163,0.799216,0.728894,0.590818,0.866970
6,No log,1.159432,0.807052,0.798666,0.807052,0.807052,0.634294,0.299595,0.968992,0.773857,0.807052,0.656449,0.428986,0.883913
7,No log,0.891173,0.794319,0.789466,0.794319,0.794319,0.707215,0.538462,0.875969,0.791612,0.794319,0.712362,0.558824,0.865900
8,No log,0.924833,0.791381,0.792837,0.791381,0.791381,0.719060,0.578947,0.859173,0.792088,0.791381,0.717552,0.573146,0.861957


TrainOutput(global_step=384, training_loss=0.41624200344085693, metrics={'train_runtime': 231.7986, 'train_samples_per_second': 132.011, 'train_steps_per_second': 2.071, 'train_loss': 0.41624200344085693, 'epoch': 8.0})

In [29]:
mbert_results = trainer.evaluate()

In [30]:
print(mbert_results)

{'eval_loss': 0.5049614906311035, 'eval_accuracy': 0.7933398628795298, 'eval_precision_weighted': 0.8126915603265202, 'eval_recall_weighted': 0.7933398628795298, 'eval_recall_micro': 0.7933398628795298, 'eval_recall_macro': 0.7589445438282647, 'eval_recall_positive': 0.6923076923076923, 'eval_recall_negative': 0.8255813953488372, 'eval_f1_weighted': 0.8002698870424636, 'eval_f1_micro': 0.7933398628795298, 'eval_f1_macro': 0.7383695017226994, 'eval_f1_positive': 0.6184448462929476, 'eval_f1_negative': 0.8582941571524514, 'eval_runtime': 2.8162, 'eval_samples_per_second': 362.55, 'eval_steps_per_second': 22.726, 'epoch': 8.0}


# **Hing-MBERT Test results**

In [31]:
test_results_mbert = trainer.predict(mbert_dataset['test'])

In [32]:
test_results_mbert

PredictionOutput(predictions=array([[ 0.97646546, -0.8913293 ],
       [-0.04362743,  0.12350859],
       [ 1.5238883 , -1.4051309 ],
       ...,
       [-0.06434073,  0.23290367],
       [-0.90692514,  1.0617005 ],
       [ 0.4490474 , -0.44061908]], dtype=float32), label_ids=array([0, 0, 0, ..., 1, 0, 0]), metrics={'test_loss': 0.5117809176445007, 'test_accuracy': 0.7864838393731636, 'test_precision_weighted': 0.8127890071188342, 'test_recall_weighted': 0.7864838393731636, 'test_recall_micro': 0.7864838393731636, 'test_recall_macro': 0.7613140633336472, 'test_recall_positive': 0.7125506072874493, 'test_recall_negative': 0.810077519379845, 'test_f1_weighted': 0.7952062839784808, 'test_f1_micro': 0.7864838393731636, 'test_f1_macro': 0.7347230167810831, 'test_f1_positive': 0.6175438596491227, 'test_f1_negative': 0.8519021739130435, 'test_runtime': 2.874, 'test_samples_per_second': 355.248, 'test_steps_per_second': 22.268})

In [33]:
test_results_mbert.predictions.argmax(axis=1)

array([0, 1, 0, ..., 1, 1, 0])

In [34]:
test_df['itags_hingMbert'] = test_results_mbert.predictions.argmax(axis=1)

In [35]:
print(test_results_mbert.metrics)

{'test_loss': 0.5117809176445007, 'test_accuracy': 0.7864838393731636, 'test_precision_weighted': 0.8127890071188342, 'test_recall_weighted': 0.7864838393731636, 'test_recall_micro': 0.7864838393731636, 'test_recall_macro': 0.7613140633336472, 'test_recall_positive': 0.7125506072874493, 'test_recall_negative': 0.810077519379845, 'test_f1_weighted': 0.7952062839784808, 'test_f1_micro': 0.7864838393731636, 'test_f1_macro': 0.7347230167810831, 'test_f1_positive': 0.6175438596491227, 'test_f1_negative': 0.8519021739130435, 'test_runtime': 2.874, 'test_samples_per_second': 355.248, 'test_steps_per_second': 22.268}


# **Hing BERT Training**

In [36]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [37]:
## dataset pre processing and alignment
# dataset = load_dataset('csv', data_files={'train': "data/final_train.csv",
#                                               'val':"data/final_val.csv",'test':"data/final_test.csv"})

bert_tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/hing-bert")
bert_tokenizer.add_tokens(special_tokens)
bert_dataset = dataset.map(
    lambda example: bert_tokenizer(example['interleaved_text_tags'], max_length=194, padding='max_length', truncation=True),
    batched=True,
    batch_size=64
)


bert_dataset = bert_dataset.remove_columns(["clean_text","language_tags","interleaved_text_tags"])
bert_dataset.set_format("torch")

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

In [38]:
## model training
bert_model = AutoModelForSequenceClassification.from_pretrained("l3cube-pune/hing-bert")

bert_model.resize_token_embeddings(len(bert_tokenizer))
arguments = TrainingArguments(
    output_dir="sample_Hing_Bert_trainer_5k_itags",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    optim = 'adamw_torch',
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=3e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1_weighted',  # Define the metric for early stopping
    greater_is_better=True,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224
)

bert_trainer = WeightedLossTrainer(
    model=bert_model,
    args=arguments,
    train_dataset=bert_dataset['train'],
    eval_dataset=bert_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=bert_tokenizer,
    compute_metrics=compute_metrics
)

bert_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.01))
bert_trainer.add_callback(LoggingCallback("sample_Hing_Bert_trainer_5k_itags/log.jsonl"))

config.json:   0%|          | 0.00/716 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/hing-bert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [39]:
# train the model
wandb.init(project="rh4", name="Hing_BERT_itag")
bert_trainer.train()

eval/accuracy,▅▁▂▇██▇▇▇
eval/f1_macro,▁▅▆██▅▇▇█
eval/f1_micro,▅▁▂▇██▇▇▇
eval/f1_negative,▇▁▂▇▇█▇▇▇
eval/f1_positive,▁▇▇██▅▇▇█
eval/f1_weighted,▁▂▃██▆▇▇█
eval/loss,▃▂▁▁▂█▅▅▁
eval/precision_weighted,▁▇██▇▇▆▇█
eval/recall_macro,▁▇██▇▄▆▇█
eval/recall_micro,▅▁▂▇██▇▇▇
eval/recall_negative,█▁▁▅▆█▆▆▅


Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,Recall Micro,Recall Macro,Recall Positive,Recall Negative,F1 Weighted,F1 Micro,F1 Macro,F1 Positive,F1 Negative
1,No log,0.690203,0.729677,0.655981,0.729677,0.729677,0.515724,0.101215,0.930233,0.673256,0.729677,0.496268,0.153374,0.839161
2,No log,0.680480,0.676787,0.682973,0.676787,0.676787,0.569051,0.360324,0.777778,0.679766,0.676787,0.567635,0.350394,0.784876
3,No log,0.621676,0.785504,0.770146,0.785504,0.785504,0.665566,0.433198,0.897933,0.774462,0.785504,0.679058,0.494226,0.863891
4,No log,0.590972,0.796278,0.788116,0.796278,0.796278,0.700237,0.514170,0.886305,0.791286,0.796278,0.709069,0.549784,0.868354
5,No log,0.540559,0.747307,0.803085,0.747307,0.747307,0.749257,0.753036,0.745478,0.762412,0.747307,0.703878,0.590476,0.817280
6,No log,0.610214,0.792360,0.783515,0.792360,0.792360,0.693519,0.502024,0.885013,0.786918,0.792360,0.702561,0.539130,0.865992
7,No log,0.634457,0.693438,0.790595,0.693438,0.693438,0.724754,0.785425,0.664083,0.715040,0.693438,0.660044,0.553495,0.766592
8,No log,0.730535,0.799216,0.797853,0.799216,0.799216,0.722850,0.574899,0.870801,0.798513,0.799216,0.724387,0.580777,0.867997


TrainOutput(global_step=384, training_loss=0.5286621252695719, metrics={'train_runtime': 207.3869, 'train_samples_per_second': 147.55, 'train_steps_per_second': 2.315, 'train_loss': 0.5286621252695719, 'epoch': 8.0})

In [43]:
bert_eval_results = bert_trainer.evaluate()
print(bert_eval_results)

{'eval_loss': 0.7305354475975037, 'eval_accuracy': 0.7992164544564153, 'eval_precision_weighted': 0.797852914052357, 'eval_recall_weighted': 0.7992164544564153, 'eval_recall_micro': 0.7992164544564153, 'eval_recall_macro': 0.7228499095084162, 'eval_recall_positive': 0.5748987854251012, 'eval_recall_negative': 0.8708010335917312, 'eval_f1_weighted': 0.7985131725557654, 'eval_f1_micro': 0.7992164544564153, 'eval_f1_macro': 0.7243872602272534, 'eval_f1_positive': 0.5807770961145196, 'eval_f1_negative': 0.8679974243399872, 'eval_runtime': 2.8697, 'eval_samples_per_second': 355.78, 'eval_steps_per_second': 22.302, 'epoch': 8.0}


# **Test results for Hing-BERT**

In [44]:
bert_results = bert_trainer.predict(bert_dataset['test'])
test_df['hingBert_itags'] = bert_results.predictions.argmax(axis=1)
print(bert_results.metrics)

{'test_loss': 0.7704779505729675, 'test_accuracy': 0.7727717923604309, 'test_precision_weighted': 0.7734004226983395, 'test_recall_weighted': 0.7727717923604309, 'test_recall_micro': 0.7727717923604309, 'test_recall_macro': 0.6916250823839563, 'test_recall_positive': 0.5344129554655871, 'test_recall_negative': 0.8488372093023255, 'test_f1_weighted': 0.7730829356047315, 'test_f1_micro': 0.7727717923604308, 'test_f1_macro': 0.6910966907315444, 'test_f1_positive': 0.5322580645161291, 'test_f1_negative': 0.8499353169469598, 'test_runtime': 2.9442, 'test_samples_per_second': 346.785, 'test_steps_per_second': 21.738}


# **Muril cased training**

In [42]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [45]:
muril_tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
print(len(muril_tokenizer))

## Adding special tokens
special_tokens = ["[EN]", "[HI]", "[OOV]"]
muril_tokenizer.add_tokens(special_tokens)
print(len(muril_tokenizer))
muril_dataset = dataset.map(
    lambda example: muril_tokenizer(example['interleaved_text_tags'],padding='max_length', truncation=True),
    batched=True,
    batch_size=16
)

muril_dataset = muril_dataset.remove_columns(["clean_text","language_tags","interleaved_text_tags"])
muril_dataset.set_format("torch")

muril_model = AutoModelForSequenceClassification.from_pretrained("google/muril-base-cased",num_labels=2)

muril_model.resize_token_embeddings(len(muril_tokenizer))

arguments = TrainingArguments(
    output_dir="sample_Hing_muril_trainer_5k_itags",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    optim = 'adafactor',
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=5e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.1,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1_weighted',  # Define the metric for early stopping
    greater_is_better=True,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224,
    report_to ="wandb"
)

muril_trainer = WeightedLossTrainer(
    model=muril_model,
    args=arguments,
    train_dataset=muril_dataset['train'],
    eval_dataset=muril_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=muril_tokenizer,
    compute_metrics=compute_metrics
)

muril_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.01))
muril_trainer.add_callback(LoggingCallback("sample_Hing_muril_trainer_5k_itags/log.jsonl"))

tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

197285
197288


Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/953M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [46]:
wandb.init(project="rh4", name="muril_itagv1")
muril_trainer.train()

eval/accuracy,▄▁▇█▅█▂███
eval/f1_macro,▁▃▇█▇▇▆███
eval/f1_micro,▄▁▇█▅█▂███
eval/f1_negative,▆▂██▄█▁███
eval/f1_positive,▁▄▆▇█▇▇███
eval/f1_weighted,▁▁▇█▆▇▃███
eval/loss,▇▆▄▃▁▄▄███
eval/precision_weighted,▁▂▆▇█▇▇███
eval/recall_macro,▁▃▅▇█▆▇▇▇▇
eval/recall_micro,▄▁▇█▅█▂███
eval/recall_negative,█▄▇▇▃▇▁▆▆▆


Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,Recall Micro,Recall Macro,Recall Positive,Recall Negative,F1 Weighted,F1 Micro,F1 Macro,F1 Positive,F1 Negative
1,No log,0.692950,0.758080,0.574686,0.758080,0.758080,0.500000,0.000000,1.000000,0.653765,0.758080,0.431198,0.000000,0.862396
2,No log,0.692639,0.758080,0.574686,0.758080,0.758080,0.500000,0.000000,1.000000,0.653765,0.758080,0.431198,0.000000,0.862396
3,0.692700,0.692703,0.758080,0.574686,0.758080,0.758080,0.500000,0.000000,1.000000,0.653765,0.758080,0.431198,0.000000,0.862396
4,0.692700,0.692522,0.758080,0.574686,0.758080,0.758080,0.500000,0.000000,1.000000,0.653765,0.758080,0.431198,0.000000,0.862396
5,0.692700,0.692897,0.758080,0.574686,0.758080,0.758080,0.500000,0.000000,1.000000,0.653765,0.758080,0.431198,0.000000,0.862396


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning

TrainOutput(global_step=960, training_loss=0.692796262105306, metrics={'train_runtime': 398.1031, 'train_samples_per_second': 115.297, 'train_steps_per_second': 7.234, 'train_loss': 0.692796262105306, 'epoch': 5.0})

In [47]:
muril_eval_results = muril_trainer.evaluate()
print(muril_eval_results)

{'eval_loss': 0.6929502487182617, 'eval_accuracy': 0.7580803134182175, 'eval_precision_weighted': 0.5746857615922628, 'eval_recall_weighted': 0.7580803134182175, 'eval_recall_micro': 0.7580803134182175, 'eval_recall_macro': 0.5, 'eval_recall_positive': 0.0, 'eval_recall_negative': 1.0, 'eval_f1_weighted': 0.6537650836609474, 'eval_f1_micro': 0.7580803134182175, 'eval_f1_macro': 0.43119777158774375, 'eval_f1_positive': 0.0, 'eval_f1_negative': 0.8623955431754875, 'eval_runtime': 6.7523, 'eval_samples_per_second': 151.208, 'eval_steps_per_second': 9.478, 'epoch': 5.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [48]:
muril_results = muril_trainer.predict(muril_dataset['test'])
test_df['muril_itags'] = muril_results.predictions.argmax(axis=1)
print(muril_results.metrics)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'test_loss': 0.6929271817207336, 'test_accuracy': 0.7580803134182175, 'test_precision_weighted': 0.5746857615922628, 'test_recall_weighted': 0.7580803134182175, 'test_recall_micro': 0.7580803134182175, 'test_recall_macro': 0.5, 'test_recall_positive': 0.0, 'test_recall_negative': 1.0, 'test_f1_weighted': 0.6537650836609474, 'test_f1_micro': 0.7580803134182175, 'test_f1_macro': 0.43119777158774375, 'test_f1_positive': 0.0, 'test_f1_negative': 0.8623955431754875, 'test_runtime': 6.6968, 'test_samples_per_second': 152.46, 'test_steps_per_second': 9.557}


## XLMR training

In [49]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [51]:
xlmr_tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")
xlmr_tokenizer.add_tokens(special_tokens)

xlmr_dataset = dataset.map(
    lambda example: xlmr_tokenizer(example['interleaved_text_tags'], max_length=194, padding='max_length', truncation=True),
    batched=True,
    batch_size=16
)

xlmr_dataset = xlmr_dataset.remove_columns(["clean_text","language_tags","interleaved_text_tags"])
xlmr_dataset.set_format("torch")

xlmr_model = AutoModelForSequenceClassification.from_pretrained("FacebookAI/xlm-roberta-base",num_labels=2)

xlmr_model.resize_token_embeddings(len(xlmr_tokenizer))

arguments = TrainingArguments(
    output_dir="sample_Hing_xlmr_trainer_5k_itags",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    optim = 'adafactor',
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy= "epoch",
    learning_rate=2e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.001,               # strength of weight decay
    load_best_model_at_end=True,
     metric_for_best_model='eval_f1_weighted',  # Define the metric for early stopping
    greater_is_better=True,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224
)

xlmr_trainer = WeightedLossTrainer(
    model=xlmr_model,
    args=arguments,
    train_dataset=xlmr_dataset['train'],
    eval_dataset=xlmr_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=xlmr_tokenizer,
    compute_metrics=compute_metrics
)

xlmr_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=5, early_stopping_threshold=0.01))
xlmr_trainer.add_callback(LoggingCallback("sample_Hing_xlmr_trainer_5k_itags/log.jsonl"))

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [52]:
import torch
torch.cuda.empty_cache()

In [53]:
wandb.init(project="rh4", name="xlmr_itag")
xlmr_trainer.train()

eval/accuracy,▁▁▁▁▁▁
eval/f1_macro,▁▁▁▁▁▁
eval/f1_micro,▁▁▁▁▁▁
eval/f1_negative,▁▁▁▁▁▁
eval/f1_positive,▁▁▁▁▁▁
eval/f1_weighted,▁▁▁▁▁▁
eval/loss,█▃▄▁▇█
eval/precision_weighted,▁▁▁▁▁▁
eval/recall_macro,▁▁▁▁▁▁
eval/recall_micro,▁▁▁▁▁▁
eval/recall_negative,▁▁▁▁▁▁


Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,Recall Micro,Recall Macro,Recall Positive,Recall Negative,F1 Weighted,F1 Micro,F1 Macro,F1 Positive,F1 Negative
1,No log,0.687577,0.758080,0.574686,0.758080,0.758080,0.500000,0.000000,1.000000,0.653765,0.758080,0.431198,0.000000,0.862396
2,No log,0.661753,0.746327,0.724047,0.746327,0.746327,0.608025,0.340081,0.875969,0.731687,0.746327,0.616536,0.393443,0.839628
3,0.694200,0.693857,0.241920,0.058525,0.241920,0.241920,0.500000,1.000000,0.000000,0.094249,0.241920,0.194795,0.389590,0.000000
4,0.694200,0.692744,0.758080,0.574686,0.758080,0.758080,0.500000,0.000000,1.000000,0.653765,0.758080,0.431198,0.000000,0.862396
5,0.694200,0.693792,0.241920,0.058525,0.241920,0.241920,0.500000,1.000000,0.000000,0.094249,0.241920,0.194795,0.389590,0.000000
6,0.698100,0.693567,0.758080,0.574686,0.758080,0.758080,0.500000,0.000000,1.000000,0.653765,0.758080,0.431198,0.000000,0.862396
7,0.698100,0.692454,0.758080,0.574686,0.758080,0.758080,0.500000,0.000000,1.000000,0.653765,0.758080,0.431198,0.000000,0.862396


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning

TrainOutput(global_step=1344, training_loss=0.696818953468686, metrics={'train_runtime': 294.1034, 'train_samples_per_second': 208.09, 'train_steps_per_second': 13.057, 'train_loss': 0.696818953468686, 'epoch': 7.0})

In [54]:
xlmr_eval_results = xlmr_trainer.evaluate()
print(xlmr_eval_results)

{'eval_loss': 0.6617525219917297, 'eval_accuracy': 0.7463271302644466, 'eval_precision_weighted': 0.72404740248674, 'eval_recall_weighted': 0.7463271302644466, 'eval_recall_micro': 0.7463271302644466, 'eval_recall_macro': 0.6080249819539905, 'eval_recall_positive': 0.340080971659919, 'eval_recall_negative': 0.875968992248062, 'eval_f1_weighted': 0.7316873395585562, 'eval_f1_micro': 0.7463271302644466, 'eval_f1_macro': 0.616535552961478, 'eval_f1_positive': 0.39344262295081966, 'eval_f1_negative': 0.8396284829721362, 'eval_runtime': 3.0008, 'eval_samples_per_second': 340.245, 'eval_steps_per_second': 21.328, 'epoch': 7.0}


In [55]:
xlmr_results = xlmr_trainer.predict(xlmr_dataset['test'])
test_df['xlmr_itags'] = xlmr_results.predictions.argmax(axis=1)
print(xlmr_results.metrics)

{'test_loss': 0.6660645008087158, 'test_accuracy': 0.7257590597453477, 'test_precision_weighted': 0.7039351204079412, 'test_recall_weighted': 0.7257590597453477, 'test_recall_micro': 0.7257590597453477, 'test_recall_macro': 0.586189310485516, 'test_recall_positive': 0.3157894736842105, 'test_recall_negative': 0.8565891472868217, 'test_f1_weighted': 0.7124703101268866, 'test_f1_micro': 0.7257590597453478, 'test_f1_macro': 0.5917259816970764, 'test_f1_positive': 0.3577981651376147, 'test_f1_negative': 0.825653798256538, 'test_runtime': 3.0139, 'test_samples_per_second': 338.759, 'test_steps_per_second': 21.235}


In [56]:
test_df.to_csv('predict_itags.csv',index=False, encoding='utf-8')